In [1]:
import pandas as pd
import numpy as np
from torchvision import datasets
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
# Define a transform to normalize the data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Download and load the test data
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# The classes are the digits 0-9
classes = ('0', '1', '2', '3', '4',
           '5', '6', '7', '8', '9')


In [3]:
# gettin the shape of the images
test,_ = next(iter(train_loader))

print(test.shape)

torch.Size([4, 1, 28, 28])


In [4]:
# building the model

import torch.nn as nn
from torch.optim import Adam

class MyConvBlock(nn.Module):
    def __init__(self):
        super(MyConvBlock, self).__init__()
        self.model = nn.Sequential(
        nn.Conv2d(1, 32,3),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
            
        nn.Conv2d(32, 64,3),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
            
        nn.Conv2d(64,128,3),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        
        nn.Flatten(),
            
        nn.Linear(128,64),
        nn.Linear(64,10)
        
        )
        

    def forward(self, x):
        return self.model(x)
    


model = MyConvBlock()
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()




In [8]:
# Training the model
num_epoch = 10
model.train()

for epoch in range(num_epoch):
    loss = 0
    correct = 0
    N = len(train_loader.dataset)  # Total number of samples

    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = criterion(output, y)
        batch_loss.backward()
        optimizer.step()

        # Accumulate the loss and correct predictions
        loss += batch_loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(y.view_as(pred)).sum().item()

    # Calculate average loss and accuracy for the epoch
    avg_loss = loss / len(train_loader)
    accuracy = correct / N

    if epoch % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epoch}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')


Epoch [1/10], Loss: 0.0235, Accuracy: 0.9940
Epoch [3/10], Loss: 0.0237, Accuracy: 0.9943
Epoch [5/10], Loss: 0.0220, Accuracy: 0.9945
Epoch [7/10], Loss: 0.0230, Accuracy: 0.9951
Epoch [9/10], Loss: 0.0247, Accuracy: 0.9954


In [10]:
# Evaluating the model using the test dataset
model.eval()
loss = 0
correct = 0
N = len(test_loader.dataset)  # Total number of samples in the test set

with torch.no_grad():
    for x, y in test_loader:
        output = model(x)

        # Calculate the batch loss and accumulate it
        batch_loss = criterion(output, y)  # Use criterion, not `loss` again
        loss += batch_loss.item()

        # Calculate the number of correct predictions
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(y.view_as(pred)).sum().item()

# Calculate average loss and overall accuracy
avg_loss = loss / len(test_loader)
accuracy = correct / N

print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(avg_loss, accuracy))


Valid - Loss: 0.1668 Accuracy: 0.9807


In [11]:
# save the model

# Specify the path where you want to save the model
model_path = "trained_model.pth"

# Save the model's state dictionary
torch.save(model.state_dict(), model_path)
print("Model saved successfully!")


Model saved successfully!
